<a href="https://colab.research.google.com/github/Jaqueline-Nascimento/BomDiaEquipamentos-LRB/blob/main/Bom_dia_equipamentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install flask
!pip install requests


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 311, in __get__
    obj = instance._get_current_object()
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 508, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 311, in __get__
    obj = instance._get_current_object()
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 508, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
  File "/usr/local/lib/py

In [1]:
from flask import Flask, render_template, request, redirect, url_for
import requests
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import requests
import logging
logging.basicConfig(level=logging.DEBUG)

app = Flask(__name__)
app.secret_key = 'your_secret_key'

# Configuração do logging
logging.basicConfig(level=logging.DEBUG)
app.logger.addHandler(logging.StreamHandler())
app.logger.setLevel(logging.DEBUG)

# Sample data for equipment quotas by management area
dados_iniciais = {
    'BG16F': {'rocadeira': 17, 'soprador': 10, 'motopoda': 1, 'motosserra': 0},
    'BG16J': {'rocadeira': 17, 'soprador': 10, 'motopoda': 1, 'motosserra': 0},
    'BG24B': {'rocadeira': 31, 'soprador': 11, 'motopoda': 1, 'motosserra': 0},
    'BG24R': {'rocadeira': 29, 'soprador': 11, 'motopoda': 1, 'motosserra': 0},
    'BGE': {'rocadeira': 20, 'soprador': 18, 'motopoda': 1, 'motosserra': 6}
}


equipamentos = {
    'Roçadeiras': {
        'oficina': 0,
        'quebradas': 0,
        'turnos': [
            {'turno': '1º Turno', 'disponiveis': 0, 'operando': 0},
            {'turno': '2º Turno', 'disponiveis': 0, 'operando': 0},
            {'turno': '3º Turno', 'disponiveis': 0, 'operando': 0}
        ]
    }
}


@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        gerencia_selecionada = request.form.get('gerencia')
    else:
        gerencia_selecionada = None

    # Log para diagnóstico
    print("Hello World")
    print("Gerência Selecionada:", gerencia_selecionada)

    quotas = dados_iniciais.get(gerencia_selecionada, {})
    print("Quotas:", quotas)

    if not quotas:
        print("Quotas: {} (Gerência não encontrada ou sem quotas)")

    # Aqui você pode renderizar a template passando as variáveis necessárias
    return render_template('index.html', quotas=quotas, equipamentos=equipamentos)








    #return render_template('index.html', quotas=dados_iniciais, equipamento=equipamentos)


@app.route('/update', methods=['POST'])
def update():
    # Retrieve selected management from form
    gerencia_selecionada = request.form['gerencia']
    cotas_gerencia = dados_iniciais.get(gerencia_selecionada, {})

    # Prepare data for POST request to Google Apps Script
    data = {
        'gerencia': gerencia_selecionada,
        'data': request.form['data'],
        'observacoes': request.form.get('observacoes', ''),
        'cota_rocadeira': cotas_gerencia.get('rocadeira', 0),
        'cota_soprador': cotas_gerencia.get('soprador', 0),
        'cota_motopoda': cotas_gerencia.get('motopoda', 0),
        'cota_motosserra': cotas_gerencia.get('motosserra', 0),
        'equipamentos': []
    }

    # Collect detailed information for each type of equipment
    for equip in ['rocadeira', 'soprador', 'motopoda', 'motosserra']:
        equip_info = {
                'oficina': request.form.get(f"oficina_{equip}", 0),
                'quebradas': request.form.get(f"quebradas_{equip}", 0),
                'disponivel': request.form.get(f"disponivel_{equip}", 0),
                'cota_rocadeira': cotas_gerencia.get('rocadeira', 0),  # Cota de roçadeiras
                'cota_soprador': cotas_gerencia.get('soprador', 0),  # Cota de sopradores
                'cota_motopoda': cotas_gerencia.get('motopoda', 0),  # Cota de motopodas
                'cota_motosserra': cotas_gerencia.get('motosserra', 0),  # Cota de motosserras
                'oficina_rocadeiras': request.form.get(f"oficina_rocadeiras_{equip}", 0),
                'oficina_sopradores': request.form.get(f"oficina_sopradores_{equip}", 0),
                'oficina_motopodas': request.form.get(f"oficina_motopodas_{equip}", 0),
                'oficina_motosserras': request.form.get(f"oficina_motosserras_{equip}", 0),
                'quebradas_rocadeiras': request.form.get(f"quebradas_rocadeiras_{equip}", 0),
                'quebradas_sopradores': request.form.get(f"quebradas_sopradores_{equip}", 0),
                'quebradas_motopodas': request.form.get(f"quebradas_motopodas_{equip}", 0),
                'quebradas_motosserras': request.form.get(f"quebradas_motosserras_{equip}", 0),
                'disponivel_rocadeiras': request.form.get(f"disponivel_rocadeiras_{equip}", 0),
                'disponivel_sopradores': request.form.get(f"disponivel_sopradores_{equip}", 0),
                'disponivel_motopodas': request.form.get(f"disponivel_motopodas_{equip}", 0),
                'disponivel_motosserras': request.form.get(f"disponivel_motosserras_{equip}", 0)
            }

        data['equipamentos'].append(equip_info)

    # URL for Google Apps Script
    url = 'https://script.google.com/macros/s/YOUR_SCRIPT_ID/exec'
    response = requests.post(url, json=data)
    if response.status_code == 200:
        print("Dados enviados com sucesso!")
    else:
        print("Falha ao enviar dados!")

    # Redirect back to the main page
    return redirect(url_for('index'))

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
